In [338]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [339]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [340]:
# Import the dependencies.
# Mandatory
from sqlalchemy import create_engine, text, inspect

# ORM imports - for classes
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

#data science
import pandas as pd
import matplotlib.pyplot as plt

# Other imports
import datetime as dt
import os

In [341]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [342]:
import os
print(os.path.isfile("./Resources/hawaii.sqlite"))


True


In [343]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)


In [344]:
# View all of the classes that automap found
print(Base.classes.keys())

['station', 'measurement']


In [345]:
# Save references to each table
table_classes = {name: getattr(Base.classes, name) for name in Base.classes.keys()}

# Print the table classes to verify
print("Table classes:")
for name, table_class in table_classes.items():
    print(f"Table Name: {name}, Class: {table_class}")

Table classes:
Table Name: station, Class: <class 'sqlalchemy.ext.automap.station'>
Table Name: measurement, Class: <class 'sqlalchemy.ext.automap.measurement'>


In [346]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

# Using the inspector to print the column names within the 'measurement' table and its types
columns = inspector.get_columns('measurement')
print("Columns in measurement table")
for column in columns:
    print(column["name"], column["type"])

# Using the inspector to print the column names within the 'station' table and its types
columns = inspector.get_columns('station')
print("\nColumns in station table")
for column in columns:
    print(column["name"], column["type"])

Columns in measurement table
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT

Columns in station table
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [347]:
# Use Pandas to calculate the summary statistics for the precipitation data

query = "SELECT prcp FROM measurement"
df = pd.read_sql(query, engine)

# Calculate summary statistics for the 'prcp' column
summary_stats = df['prcp'].describe()

# Print summary statistics
print("Summary statistics for precipitation data:")
print(summary_stats)

# Additional statistics if needed
median_precipitation = df['prcp'].median()
std_dev_precipitation = df['prcp'].std()
percentiles = df['prcp'].quantile([0.25, 0.5, 0.75])

print("\nAdditional Statistics:")
print(f"Median: {median_precipitation}")
print(f"Standard Deviation: {std_dev_precipitation}")
print("Percentiles:")
print(percentiles)


Summary statistics for precipitation data:
count    18103.000000
mean         0.160644
std          0.468746
min          0.000000
25%          0.000000
50%          0.010000
75%          0.110000
max         11.530000
Name: prcp, dtype: float64

Additional Statistics:
Median: 0.01
Standard Deviation: 0.4687457319225193
Percentiles:
0.25    0.00
0.50    0.01
0.75    0.11
Name: prcp, dtype: float64


In [348]:
# Calculate summary statistics for the 'prcp' column
summary_stats = df['prcp'].describe()

# Print summary statistics
print("Summary statistics for precipitation data:")
print(summary_stats)

# Additional statistics if needed
median_precipitation = df['prcp'].median()
std_dev_precipitation = df['prcp'].std()
percentiles = df['prcp'].quantile([0.25, 0.5, 0.75])

print("\nAdditional Statistics:")
print(f"Median: {median_precipitation}")
print(f"Standard Deviation: {std_dev_precipitation}")
print("Percentiles:")
print(percentiles)


Summary statistics for precipitation data:
count    18103.000000
mean         0.160644
std          0.468746
min          0.000000
25%          0.000000
50%          0.010000
75%          0.110000
max         11.530000
Name: prcp, dtype: float64

Additional Statistics:
Median: 0.01
Standard Deviation: 0.4687457319225193
Percentiles:
0.25    0.00
0.50    0.01
0.75    0.11
Name: prcp, dtype: float64


# Exploratory Station Analysis

In [349]:
# Design a query to calculate the total number of stations in the dataset
station = Base.classes.station

# Query the total number of distinct stations
station_count = session.query(station.id).distinct().count()
print("Total number of unique stations in the dataset:", station_count)

Total number of unique stations in the dataset: 9


In [350]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
active_stations = session.query(measurement.station, func.count(measurement.station)).\
    group_by(measurement.station).\
    order_by(func.count(measurement.station).desc()).all()
active_stations


[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [351]:
# Design a query to find the most active stations (i.e. which stations have the most rows?). But this time using SQL query
# List the stations and their counts in descending order.
query = """ 
        SELECT station, COUNT(station) AS station_count
        FROM measurement
        GROUP BY station
        ORDER BY station_count DESC;
        """
# Save the query results as a Pandas DataFrame
active_stations_sql = pd.read_sql(text(query), con = engine)
active_stations_sql


,station,station_count
0,USC00519281,2772
1,USC00519397,2724
2,USC00513117,2709
3,USC00519523,2669
4,USC00516128,2612
5,USC00514830,2202
6,USC00511918,1979
7,USC00517948,1372
8,USC00518838,511


# Close Session

In [352]:
# Close Session
session.close()